In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
get_ipython().run_line_magic('matplotlib', 'inline')
import tensorflow as tf
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
from sklearn.feature_extraction.text import CountVectorizer
import io
import time
from io import StringIO
import re
from nltk.corpus import stopwords
from PIL import *
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from ipywidgets import FileUpload
from IPython.display import display

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\siwar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
upload = FileUpload(accept='.csv', multiple=True)
display(upload)

FileUpload(value={}, accept='.csv', description='Upload', multiple=True)

In [5]:
df1=pd.read_csv(r'C:\Users\siwar\Downloads\datasetsMedical (1)\datasetsMedical\h1n1\708381_1340787_bundle_archive\data.csv')

df1.head(10)

,Date,Country,Cumulative no. of cases,Cumulative no. of deaths,Link
0,2009-04-24,Mexico,18,0,https://www.who.int/csr/don/2009_04_24/en/
1,2009-04-24,United States of America,7,0,https://www.who.int/csr/don/2009_04_24/en/
2,2009-04-26,Mexico,18,0,https://www.who.int/csr/don/2009_04_24/en/
3,2009-04-26,United States of America,20,0,https://www.who.int/csr/don/2009_04_24/en/
4,2009-04-27,Canada,6,0,https://www.who.int/csr/don/2009_04_24/en/
5,2009-04-27,Mexico,26,7,https://www.who.int/csr/don/2009_04_24/en/
6,2009-04-27,Spain,1,0,https://www.who.int/csr/don/2009_04_24/en/
7,2009-04-27,United States of America,40,0,https://www.who.int/csr/don/2009_04_24/en/
8,2009-04-28,Canada,6,0,https://www.who.int/csr/don/2009_04_24/en/
9,2009-04-28,Israel,2,0,https://www.who.int/csr/don/2009_04_24/en/


In [6]:
df1.drop(['Date', 'Country'],axis=1,inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490 entries, 0 to 2489
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Cumulative no. of cases   2490 non-null   int64 
 1   Cumulative no. of deaths  2490 non-null   int64 
 2   Link                      2490 non-null   object
dtypes: int64(2), object(1)
memory usage: 58.5+ KB


In [7]:
df1['label'] = 'conjunctivitis'
df1.head()

,Cumulative no. of cases,Cumulative no. of deaths,Link,label
0,18,0,https://www.who.int/csr/don/2009_04_24/en/,conjunctivitis
1,7,0,https://www.who.int/csr/don/2009_04_24/en/,conjunctivitis
2,18,0,https://www.who.int/csr/don/2009_04_24/en/,conjunctivitis
3,20,0,https://www.who.int/csr/don/2009_04_24/en/,conjunctivitis
4,6,0,https://www.who.int/csr/don/2009_04_24/en/,conjunctivitis


In [8]:
df1 = df1.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)# replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing.
    #text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df1['Link'] = df1['Link'].apply(clean_text)
df1['Link'] = df1['Link'].str.replace('\d+', '')

<ipython-input-8-7898b6de79b7>:19: FutureWarning:

The default value of regex will change from True to False in a future version.



In [9]:
df1.head(5)

,Cumulative no. of cases,Cumulative no. of deaths,Link,label
0,18,0,https wwwwhoint csr __ en,conjunctivitis
1,7,0,https wwwwhoint csr __ en,conjunctivitis
2,18,0,https wwwwhoint csr __ en,conjunctivitis
3,20,0,https wwwwhoint csr __ en,conjunctivitis
4,6,0,https wwwwhoint csr __ en,conjunctivitis


In [10]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [11]:
from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""

    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight


    def set_stopwords(self, stopwords):
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True

    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences

    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab

    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs

    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())

    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1

        # Get Symmeric matrix
        g = self.symmetrize(g)

        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm

        return g_norm


    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break


    def analyze(self, text,
                candidate_pos=['NOUN', 'PROPN'],
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""

        # Set stop words
        self.set_stopwords(stopwords)

        # Pare text by spaCy
        doc = nlp(text)

        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words

        # Build vocabulary
        vocab = self.get_vocab(sentences)

        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)

        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)

        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))

        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]

        self.node_weight = node_weight

In [12]:
docs = df1.text.values.tolist()

AttributeError: 'DataFrame' object has no attribute 'text'

In [ ]:
str1 = ''.join(docs)

In [ ]:
text = str1
tr4w = TextRank4Keyword()
tr4w.analyze(text, window_size=4, lower=False)
tr4w.get_keywords()

In [ ]:

get_ipython().system('pip install yake')

In [ ]:
import yake

text = str1

# assuming default parameters
simple_kwextractor = yake.KeywordExtractor()
keywords = simple_kwextractor.extract_keywords(text)

for kw in keywords:
	print(kw)

# specifying parameters
max_ngram_size = 1
custom_kwextractor = yake.KeywordExtractor(lan="en", n = max_ngram_size, dedupLim=0.9, dedupFunc='seqm', windowsSize=1,  features=None)

keywords = custom_kwextractor.extract_keywords(text)

for kw in keywords:
	print(kw)




